## LOGISTIC REGRESSION ON A LARGE DATA SET

In [22]:
import tensorflow as tf
import numpy as np
import time
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 1. Generate synthetic large dataset
N_SAMPLES = 500000     # 5 million rows
N_FEATURES = 200          # 200 features
N_CLASSES = 2             # Binary classification

# Generate synthetic data
X, y = make_classification(n_samples=N_SAMPLES,
                           n_features=N_FEATURES,
                           n_informative=150,
                           n_redundant=25,
                           n_classes=N_CLASSES,
                           random_state=42)

# Scale the features
X = StandardScaler().fit_transform(X)

# Convert to float32 for GPU efficiency
X = X.astype(np.float32)
y = y.astype(np.float32)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Define function to train and evaluate model
def run_logistic_regression(device_name):
    with tf.device(device_name):
        # Define logistic regression model
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],))
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        print(f"Running on {device_name}...")

        # Start training timer
        start_train = time.time()
        model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
        train_time = time.time() - start_train

        # Start inference timer
        start_test = time.time()
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        test_time = time.time() - start_test

        # Print results
        print(f"  Accuracy       : {acc:.4f}")
        print(f"  Training Time  : {train_time:.4f} s")
        print(f"  Inference Time : {test_time:.4f} s\n")

# 3. Run on CPU
run_logistic_regression("/CPU:0")

# 4. Run on GPU (if available)
if tf.config.list_physical_devices("GPU"):
    run_logistic_regression("/GPU:0")
else:
    print("No GPU available.")


Running on /CPU:0...
  Accuracy       : 0.8428
  Training Time  : 76.1190 s
  Inference Time : 3.2941 s

Running on /GPU:0...
  Accuracy       : 0.8423
  Training Time  : 109.4842 s
  Inference Time : 4.6298 s



## CONVOLUTIONAL NEURAL NETWORK USING GPU AND CPU

In [23]:
import tensorflow as tf
import time
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# 1. Load and preprocess CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# 2. Define CNN model
def create_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu'),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def run_cifar(device_name):
    with tf.device(device_name):
        model = create_model()
        print(f"\nRunning on {device_name}...")

        start_train = time.time()
        model.fit(x_train, y_train, epochs=10, batch_size=128, verbose=0)
        train_time = time.time() - start_train

        start_test = time.time()
        loss, acc = model.evaluate(x_test, y_test, verbose=0)
        test_time = time.time() - start_test

        print(f"  Accuracy       : {acc:.4f}")
        print(f"  Training Time  : {train_time:.2f} s")
        print(f"  Inference Time : {test_time:.2f} s")


run_cifar("/CPU:0")

if tf.config.list_physical_devices("GPU"):
    run_cifar("/GPU:0")
else:
    print("No GPU available.")

170498071/170498071 [==============================] - 2s 0us/step

Running on /CPU:0...
  Accuracy       : 0.7168
  Training Time  : 157.57 s
  Inference Time : 2.14 s

Running on /GPU:0...
  Accuracy       : 0.7119
  Training Time  : 45.62 s
  Inference Time : 2.50 s
